In [ ]:

import requests
import time
import pandas as pd

url = 'https://api.map.baidu.com/place/v2/search?query=ATM&location=23.120791,113.27267&radius=5000&output=json&ak=changetourjason&page_size=20&page_num=1'
page=0
poi_all=pd.DataFrame()
while page<20:
    url_next=url.replace('page_num=1','page_num='+str(page))
    poi=requests.get(url_next)
    poi_text=eval(poi.text)
    if poi_text['results']==[]:
        break
    else:
        poi_d=pd.DataFrame(poi_text['results'])
        poi_all=pd.concat([poi_all,poi_d])
    page+=1
    time.sleep(3)
    print(page)

poi_all.to_excel('D:\海珠广场ATM.xlsx','Sheet1',index=False)

1
2
3
4
5
6
7


In [ ]:
import pandas as pd
import math
from ast import literal_eval
import os
import folium

# ====================== 1. 读取原始数据 ======================
data=pd.read_excel(r"D:\海珠广场ATM.xlsx",'Sheet1')
output_dir = r"D:\dailydocs\人工智能与商业智能综合实验"
output_file = os.path.join(output_dir, "海珠广场ATM_GCJ02.xlsx")
data.to_excel(output_file)

# ====================== 2. BD-09转GCJ-02转换函数 ======================
def bd09_to_gcj02(lng, lat):
    """
    百度BD-09坐标系 → 火星GCJ-02坐标系
    返回: (gcj_lng, gcj_lat)
    """
    x_pi = math.pi * 3000.0 / 180.0
    x = lng - 0.0065
    y = lat - 0.006
    z = math.sqrt(x*x + y*y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gcj_lng = z * math.cos(theta)
    gcj_lat = z * math.sin(theta)
    return gcj_lng, gcj_lat

# ====================== 3. 数据转换 ======================
# 提取BD-09原始坐标
data['bd09_lat'] = data['location'].apply(lambda x: literal_eval(str(x))['lat'])
data['bd09_lng'] = data['location'].apply(lambda x: literal_eval(str(x))['lng'])
# 转换为GCJ-02
data['gcj02_lng'], data['gcj02_lat'] = zip(*data.apply(
    lambda row: bd09_to_gcj02(row['bd09_lng'], row['bd09_lat']), axis=1))
print(data[['bd09_lng', 'bd09_lat', 'gcj02_lng', 'gcj02_lat']].head())


# ====================== 4. 地图展示 ======================
# 海珠广场高德地图的坐标
center_gcj = [23.11445, 113.266173]  
# 高德地图坐标系地面图
m = folium.Map(
    location=center_gcj, 
    zoom_start=14,
    tiles='https://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    attr='高德地图',
    control_scale=True
)

# 添加距离圈
for r in [1, 2, 3, 4, 5]:
    folium.Circle(
        location=center_gcj,
        radius=r*1000,
        color='gray',
        fill=True,
        fill_opacity=0.1,
        popup=f'{r}公里范围'
    ).add_to(m)

# 标记ATM
for _, row in data.iterrows():
    folium.CircleMarker(
        location=[row['gcj02_lat'], row['gcj02_lng']],
        radius=6,
        color='pink',
        fill=True,
        popup=row.get('name', 'ATM')
    ).add_to(m)

# 标记中心点（红色）
folium.Marker(
    center_gcj,
    icon=folium.Icon(color='red'),
    popup='中心点'
).add_to(m)

display(m)
m.save(r"D:\ATM分布_火星坐标.html")
print("地图已保存，请检查HTML文件")

     bd09_lng   bd09_lat   gcj02_lng  gcj02_lat
0  113.271485  23.121861  113.265048  23.115624
1  113.273372  23.122877  113.266941  23.116617
2  113.268917  23.123000  113.262474  23.116797
3  113.277143  23.121771  113.270719  23.115474
4  113.267996  23.120223  113.261549  23.114033


地图已保存，请检查HTML文件


In [5]:
import pandas as pd
import math
from pyecharts.charts import Bar
from pyecharts import options as opts
from geopy.distance import geodesic

# ====================== 1. 数据准备 ======================
# 读取数据
file_path = r"D:\dailydocs\人工智能与商业智能综合实验\海珠广场ATM_GCJ02.xlsx"
data = pd.read_excel(file_path)

# 解析location列
def parse_location(loc_str):
    try:
        loc_dict = literal_eval(str(loc_str))
        return loc_dict['lat'], loc_dict['lng']
    except:
        return None, None

data['lat'], data['lng'] = zip(*data['location'].apply(parse_location))
data = data.dropna(subset=['lat', 'lng'])

# ====================== 2. 计算距离并分类 ======================
# 海珠广场高德地图坐标
center_point = (23.11445, 113.266173)  # 纬度, 经度

def calculate_distance(row):
    """计算ATM与海珠广场的实地距离（单位：km）"""
    atm_point = (row['lat'], row['lng'])
    return geodesic(center_point, atm_point).km

# 添加距离列
data['distance_km'] = data.apply(calculate_distance, axis=1)

# 按距离区间分类
bins = [0, 1, 2, 3, 4, 5]
labels = ['0-1km', '1-2km', '2-3km', '3-4km', '4-5km']
data['distance_range'] = pd.cut(data['distance_km'], bins=bins, labels=labels)

# 统计各区间数量
distance_counts = data['distance_range'].value_counts().sort_index()

# ====================== 3. Pyecharts柱状图 ======================
bar = (
    Bar()
    .add_xaxis(distance_counts.index.tolist())
    .add_yaxis("ATM数量", distance_counts.values.tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="ATM距离分布统计"),
        xaxis_opts=opts.AxisOpts(name="距离区间"),
        yaxis_opts=opts.AxisOpts(name="数量"),
        toolbox_opts=opts.ToolboxOpts()  
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(position="top"),
        itemstyle_opts=opts.ItemStyleOpts(color="#FF99FF") 
        )  
)

# 显示图表（Jupyter内联）
display(bar.render_notebook())

# 保存为HTML文件
bar.render(r"D:\ATM距离分布统计.html")
print("统计图表已保存至: D:\ATM距离分布统计.html")

# 输出统计结果
print("\n=== 距离分布统计 ===")
print(distance_counts)

统计图表已保存至: D:\ATM距离分布统计.html

=== 距离分布统计 ===
0-1km    23
1-2km    47
2-3km    46
3-4km     2
4-5km     4
Name: distance_range, dtype: int64


In [ ]:
import pandas as pd
import math
from pyecharts.charts import Bar
from pyecharts import options as opts
from geopy.distance import geodesic

# ====================== 1. 数据准备 ======================
# 读取数据
file_path = r"D:\dailydocs\人工智能与商业智能综合实验\海珠广场ATM_GCJ02.xlsx"
data = pd.read_excel(file_path)

# 解析location列
def parse_location(loc_str):
    try:
        loc_dict = literal_eval(str(loc_str))
        return loc_dict['lat'], loc_dict['lng']
    except:
        return None, None

data['lat'], data['lng'] = zip(*data['location'].apply(parse_location))
data = data.dropna(subset=['lat', 'lng'])

# ====================== 2. 计算方位角 ======================
# 海珠广场坐标
center_lat, center_lng = 23.11445, 113.266173

def calculate_bearing(row):
    """计算ATM相对于海珠广场的方位角（0-360度）"""
    lat1, lon1 = math.radians(center_lat), math.radians(center_lng)
    lat2, lon2 = math.radians(row['lat']), math.radians(row['lng'])
    
    dlon = lon2 - lon1
    x = math.sin(dlon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dlon)
    bearing = math.degrees(math.atan2(x, y))
    return (bearing + 360) % 360  # 转换为0-360度

# 添加方位角列
data['bearing'] = data.apply(calculate_bearing, axis=1)

# 按方位角分类
bearing_bins = [0, 90, 180, 270, 360]
bearing_labels = ['0-90°', '90-180°', 
                 '180-270°', '270-360°']
data['bearing_range'] = pd.cut(data['bearing'], bins=bearing_bins, labels=bearing_labels)

# 统计各方位数量
bearing_counts = data['bearing_range'].value_counts().sort_index()

# ====================== 3. Pyecharts柱状图 ======================
bar = (
    Bar()
    .add_xaxis(bearing_counts.index.tolist())
    .add_yaxis("ATM数量", bearing_counts.values.tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="ATM方位角分布统计"),
        xaxis_opts=opts.AxisOpts(name="方位区间", axislabel_opts=opts.LabelOpts(rotate=-15)),
        yaxis_opts=opts.AxisOpts(name="数量"),
        toolbox_opts=opts.ToolboxOpts()
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(position="top"),
        itemstyle_opts=opts.ItemStyleOpts(color="#99FFCC") 
    )
)

# 显示图表（Jupyter内联）
display(bar.render_notebook())

# 保存为HTML文件
bar.render(r"D:\ATM方位角分布统计.html")
print("方位角统计图表已保存至: D:\ATM方位角分布统计.html")

# 输出统计结果
print("\n=== 方位角分布统计 ===")
print(bearing_counts)

方位角统计图表已保存至: D:\ATM方位角分布统计.html

=== 方位角分布统计 ===
0-90°       70
90-180°     21
180-270°     5
270-360°    27
Name: bearing_range, dtype: int64
